In [116]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [125]:
# train_loader = torch.utils.data.DataLoader(trainset_new, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True)

In [126]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))
trainset_unlabeled_import = pickle.load(open("../data/kaggle/train_unlabeled.p", "rb"))
trainset_unlabeled_import.train_labels = [10 for i in range(len(trainset_unlabeled_import.train_data))]
train_dataset_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)
unlabelled_dataset_loader = torch.utils.data.DataLoader(trainset_unlabeled_import, batch_size=64, shuffle=True)

In [127]:
class ConvVAE(nn.Module):
    def __init__(self):
        super(ConvVAE, self).__init__()
        self.latent_vars = 150
        
        self.fc11 = nn.Linear(64*3*3, 300)
        self.fc12 = nn.Linear(128, 64)
        
        
        self.conv1 = nn.Conv2d(1, 16, 2, 2, 1, 1, 1, 1)
        self.conv2 = nn.Conv2d(16, 32, 2, 2, 1, 1, 1, 1)
        self.conv3 = nn.Conv2d(32, 64, 2, 2, 1, 1, 1, 1)
    
        self.dconv1 = nn.ConvTranspose2d(16, 16, 3,3,2,2)
        self.dconv2 = nn.ConvTranspose2d(16, 32, 3,3,2,2)
        self.dconv3 = nn.ConvTranspose2d(32, 1, 3,3,2,2)
        
        
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)

        self.fc3 = nn.Linear(150, 64)
        #self.fc4 = nn.Linear(400, 784)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()


    def encode(self, x):
        #print("Input size" , x.size())
        x = self.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.conv3(x))
        #print("Size after encoding", x.size())
        x = x.view(-1, 64*3*3)
        #print(x.size())
        z = self.fc11(x)
        mu = z[:, 0:self.latent_vars]
        log_sig = z[:, self.latent_vars:]
        return mu, log_sig

    def reparametrize(self, mu, logvar):
        eps = Variable(torch.randn(logvar.size()))
        z = mu + torch.exp(logvar / 2) * eps
        
        return z

    def decode(self, x):
        x = F.relu(self.fc3(x))
        x = x.view(-1,16, 2,2)
        x = self.bn1(self.dconv1(x))
        x = self.bn2(self.dconv2(x))
        x = self.dconv3(x)
        return self.sigmoid(x)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


In [128]:
class DenoiseAE(nn.Module):
    def __init__(self):
        super(DenoiseAE, self).__init__()
        self.latent_vars = 10
        
        self.fc10 = nn.Linear(64, 10)
        self.fc11 = nn.Linear(64, 10)
        self.fc12 = nn.Linear(128, 64)
        
        self.conv1 = nn.Conv2d(1, 32, 3, 3, 1, 1, 1, 1)
        self.conv2 = nn.Conv2d(32, 16, 3, 3, 1, 1, 1, 1)
        self.conv3 = nn.Conv2d(16, 16, 3, 3, 1, 1, 1, 1)
        
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.dconv1 = nn.ConvTranspose2d(16, 16, 3,3,2,2)
        self.dconv2 = nn.ConvTranspose2d(16, 32, 3,3,2,2)
        self.dconv3 = nn.ConvTranspose2d(32, 1, 3,3,2,2)
        

        self.fc3 = nn.Linear(10, 64)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = x.view(-1, 16*2*2)
        z = self.fc11(x)
        mu = self.fc10(x)
        log_sig = self.fc11(x)
        return mu, log_sig

    def reparametrize(self, mu, logvar):
        eps = Variable(torch.randn(logvar.size()))
        z = mu + torch.exp(logvar / 2) * eps
        return z

    def decode(self, x):
       
        x = F.relu(self.fc3(x))
        x = x.view(-1,16, 2,2)
        x = self.dconv1(x)
        x = self.dconv2(x)
        x = self.dconv3(x)
        
        return self.sigmoid(x)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [129]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.classifyBN = NetBN()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.conv1_bn = nn.BatchNorm2d(10)
        self.conv2_bn = nn.BatchNorm2d(20)
        self.conv3_bn = nn.BatchNorm2d(256)
        
        self.classfc1 = nn.Linear(320, 160)
        self.classfc2 = nn.Linear(160, 10)
        self.classfc3 = nn.Linear(80, 10)
        
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 10)
        self.fc22 = nn.Linear(400, 10)
        self.fc3 = nn.Linear(10, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if False:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def classify(self, x):
       
        x = x.view(-1, 1, 28,28)
        x = F.relu(F.max_pool2d(self.conv1_bn(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2_bn(self.conv2(x))), 2))

        x = x.view(-1, 320)
        x = F.relu(self.classfc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.classfc2(x))
        return F.log_softmax(x)
        
    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar,logvar


model = VAE()
input = Variable(torch.Tensor(1,64,28,28))


In [130]:
reconstruction_function = nn.MSELoss()
reconstruction_function.size_average = False

model = VAE()
def loss_function(recon_x, x, mu, logvar):
    BCE = reconstruction_function(recon_x, x)

    # see Appendix B froms VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    #print("Reconstruction loss", BCE)
    #print("KL div loss", KLD)
    return BCE + KLD




In [131]:
# CPU only training
def generate_train(epoch, data_loader, isSupervised):
    model.train()
    train_loss = 0
    pred = 0
    correct  = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        recon_batch, mu, logvar, output = model(data)
        #output = F.log_softmax(output)
        class_loss = 0
        if isSupervised:
            class_loss = F.nll_loss(output, target)
        loss = loss_function(recon_batch, data, mu, logvar) +  class_loss        
        loss.backward()
        
        train_loss += loss.data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(data_loader.dataset)))
    print('Training accuracy {}'.format(correct/len(data_loader.dataset)))
    return correct/len(data_loader.dataset)
    
def generate_test(epoch, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target, volatile=True)
        recon_batch, mu, logvar, output = model(data)
        #output = F.log_softmax(output)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct/len(test_loader.dataset)


In [132]:
train_acc = []
val_acc = []

In [ ]:
#Generative phase
model = VAE()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 40):
    generate_train(epoch, unlabelled_dataset_loader , False)
    train_acc.append(100*generate_train(epoch, train_loader, True))
    val_acc.append(100*generate_test(epoch, valid_loader))
    generate_images()

In [ ]:
def plot(train_errors, test_errors, title):
    import matplotlib.pyplot as plt
    import numpy as np
    plt.plot(np.arange(len(train_errors)),train_errors,label="Training accuracy")
    plt.plot(np.arange(len(train_errors)),test_errors,label="Va accuracy")
    plt.xlabel('Epoch size')
    legend = plt.legend(loc='lower center', shadow=True)
    plt.ylabel('Accuracy Value')
    plt.title(title)
    plt.savefig(title+'.jpg')
    plt.show()
plot(train_acc, val_acc, 'VAE Classifier with unlabelled - Approach 1')

In [ ]:
# Check generated images
def generate_images():
    from  torchvision.utils import save_image
    for index, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        gen_data, mu, logvar, _ = model(data)
        gen_images = gen_data.view(-1, 1,28,28)
        #print(data[0])
        save_image(gen_images.data, 'images/1gen{}.jpg'.format(index))
        save_image(data.data, 'images/1orgin{}.jpg'.format(index))
    print("Images generated")